In [1]:
# Load necessary library and module
import warnings
warnings.filterwarnings('ignore')

import sys
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
import string
import unicodedata
import nltk
import sklearn

nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

%matplotlib inline
# %load_ext google.colab.data_table

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
mlbb = pd.read_csv('dataset/mlbb_clean_content_after_preprocessing_20062022_noneutral.csv',encoding='utf-8')
# ffm = pd.read_csv('dataset/ffm_clean_content_after_preprocessing_20062022.csv')#, encoding='ISO-8859-1')
# hdi = pd.read_csv('dataset/hdi_clean_content_after_preprocessing_20062022.csv')#, encoding='ISO-8859-1')

In [6]:
mlbb = mlbb[['clean_content','score','sentiment','polarity']]
# ffm = ffm[['content', 'clean_content','score','sentiment','polarity']]
# hdi = hdi[['content', 'clean_content','score','sentiment','polarity']]
#mlbb.head()
#ffm.head()

In [7]:
cat_category = [feature for feature in mlbb.columns if mlbb[feature].dtypes == "O"]
mlbb[cat_category].isnull().sum()

clean_content    422
polarity         400
dtype: int64

In [6]:
# cat_category = [feature for feature in ffm.columns if ffm[feature].dtypes == "O"]
# ffm[cat_category].isnull().sum()

content           0
clean_content    66
polarity          0
dtype: int64

In [7]:
# cat_category = [feature for feature in hdi.columns if hdi[feature].dtypes == "O"]
# hdi[cat_category].isnull().sum()

content            0
clean_content    101
polarity           0
dtype: int64

In [8]:
mlbb.isnull().sum()#.values()

clean_content    422
score            400
sentiment        400
polarity         400
dtype: int64

# Remove NULL Values

In [9]:
mlbb= mlbb[mlbb['clean_content'].notna()]
mlbb['clean_content'].dropna(inplace=True)
# ffm= ffm[ffm['clean_content'].notna(inplace=True)]
# hdi= hdi[hdi['clean_content'].notna(inplace=True)]

In [12]:
# mlbb.shape
mlbb.isnull().sum()#.values()

clean_content    0
score            0
sentiment        0
polarity         0
dtype: int64

In [ ]:
# ffm.shape

In [ ]:
# hdi.shape

In [13]:
mlbb.info()
mlbb.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 778 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clean_content  778 non-null    object 
 1   score          778 non-null    float64
 2   sentiment      778 non-null    float64
 3   polarity       778 non-null    object 
dtypes: float64(2), object(2)
memory usage: 30.4+ KB


,clean_content,score,sentiment,polarity
0,rate higher problem everytime hold heros abili...,1.0,0.0,negative
1,recently even start game always crashed hero s...,1.0,0.0,negative
2,game interesting heroes recently game bugs man...,1.0,0.0,negative
3,years playing game never went previously whene...,1.0,0.0,negative
4,know care anymore developer never played aroun...,1.0,0.0,negative


In [14]:
mlbb.score.value_counts()


1.0    199
2.0    195
5.0    195
4.0    189
Name: score, dtype: int64

In [ ]:
# ffm.score.value_counts()

In [ ]:
# hdi.score.value_counts()

In [15]:
mlbb.sentiment.value_counts()

0.0    394
2.0    384
Name: sentiment, dtype: int64

In [ ]:
# ffm.sentiment.value_counts()

In [ ]:
# hdi.sentiment.value_counts()

In [16]:
class_names = ['negative', 'positive']

# Word Lenght

In [18]:
from tqdm._tqdm_notebook import tqdm_notebook

In [19]:
tqdm_notebook.pandas()
def count_words(x):
    words = word_tokenize(x)
    n=len(words)
    return n

In [20]:
mlbb['word_length'] = mlbb['clean_content'].progress_apply(str).progress_apply(lambda x:count_words(x))

  0%|          | 0/778 [00:00<?, ?it/s]

  0%|          | 0/778 [00:00<?, ?it/s]

In [ ]:
# ffm['word_length'] = ffm['clean_content'].progress_apply(str).progress_apply(lambda x:count_words(x))

In [ ]:
# hdi['word_length'] = hdi['clean_content'].progress_apply(str).progress_apply(lambda x:count_words(x))

In [21]:
mlbb['word_length'].value_counts().sort_index()

1     73
2     67
3     44
4     23
5     23
6     14
7     14
8     10
9     17
10     8
11     7
12     5
13     7
14    10
15    10
16     8
17    17
18    14
19    14
20    12
21    14
22    16
23    15
24    16
25    14
26    14
27    13
28    16
29    18
30    17
31    21
32    21
33    13
34    22
35    12
36    13
37    21
38     8
39    12
40    12
41    18
42    18
43     9
44    13
45     4
46     3
47     2
49     1
51     1
52     2
53     1
58     1
Name: word_length, dtype: int64

In [ ]:
# ffm['word_length'].value_counts().sort_index()

In [ ]:
# hdi['word_length'].value_counts().sort_index()

In [22]:
#sama dengan dropna
mlbb = mlbb.drop(mlbb[mlbb['word_length']==0].index,axis=0)
mlbb = mlbb.reset_index(drop=True)
# ffm = ffm.drop(ffm[ffm['word_length']==0].index,axis=0)
# ffm = ffm.reset_index(drop=True)
# hdi = hdi.drop(hdi[hdi['word_length']==0].index,axis=0)
# hdi = hdi.reset_index(drop=True)

In [23]:
mlbb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   clean_content  778 non-null    object 
 1   score          778 non-null    float64
 2   sentiment      778 non-null    float64
 3   polarity       778 non-null    object 
 4   word_length    778 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 30.5+ KB


# Word Tokenize

In [24]:
mlbb_word_dict = {}
for i in range(0,len(mlbb['clean_content'])):
    sentence = mlbb['clean_content'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in mlbb_word_dict:
            mlbb_word_dict[j] = 1
        else:
            mlbb_word_dict[j] += 1

In [ ]:
ffm_word_dict = {}
for i in range(0,len(ffm['clean_content'])):
    sentence = ffm['clean_content'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in ffm_word_dict:
            ffm_word_dict[j] = 1
        else:
            ffm_word_dict[j] += 1

In [ ]:
hdi_word_dict = {}
for i in range(0,len(hdi['clean_content'])):
    sentence = hdi['clean_content'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in hdi_word_dict:
            hdi_word_dict[j] = 1
        else:
            hdi_word_dict[j] += 1

In [ ]:
len(mlbb_word_dict)
print(mlbb_word_dict)
mlbb_wd=pd.Series(mlbb_word_dict)
mlbb_worddict_df = pd.DataFrame({'word':mlbb_wd.index, 'count':mlbb_wd.values})
# mlbb_worddict_df.head()
mlbb_worddict_df.to_csv( "dataset/mlbb_worddict_clean_content_20062022.csv", index=False, encoding='utf-8-sig')

In [ ]:
len(ffm_word_dict)
print(ffm_word_dict)
ffm_wd=pd.Series(ffm_word_dict)
ffm_worddict_df = pd.DataFrame({'word':ffm_wd.index, 'count':ffm_wd.values})
# ffm_worddict_df.head()
ffm_worddict_df.to_csv( "dataset/ffm_worddict_clean_content_20062022.csv", index=False, encoding='utf-8-sig')

In [29]:
len({k:v for (k,v) in mlbb_word_dict.items() if v < 4})

2281

In [ ]:
len({k:v for (k,v) in toped_word_dict.items() if v < 4})

In [ ]:
len({k:v for (k,v) in lazada_word_dict.items() if v < 4})

# Wordcloud

In [30]:
from wordcloud import WordCloud

In [ ]:
shopee_word_to_plot = shopee['clean_content'].copy()
shopee_word_to_plot.head()

In [ ]:

shopee_word_to_plot_df=pd.DataFrame(shopee_word_to_plot)
# print(shopee_word_to_plot)
shopee_word_to_plot_df.head()
shopee_wordcloud = WordCloud(width = 1024, height = 1024, background_color = 'white', max_words = 1200
                      , min_font_size = 20).generate(str(shopee_word_to_plot))
#plot the word cloud
fig = plt.figure(figsize = (8,8), facecolor = None)
plt.imshow(shopee_wordcloud)
plt.axis('off')
plt.show()

In [ ]:
toped_word_to_plot = toped['clean_content'].copy()
toped_word_to_plot_df=pd.DataFrame(toped_word_to_plot)
# print(toped_word_to_plot)
toped_word_to_plot_df.head()
toped_wordcloud = WordCloud(width = 1024, height = 1024, background_color = 'white', max_words = 1200
                      , min_font_size = 20).generate(str(toped_word_to_plot))
#plot the word cloud
fig = plt.figure(figsize = (8,8), facecolor = None)
plt.imshow(toped_wordcloud)
plt.axis('off')
plt.show()

In [ ]:
lazada_word_to_plot = lazada['clean_content'].copy()
lazada_word_to_plot_df=pd.DataFrame(lazada_word_to_plot)
# print(lazada_word_to_plot)
lazada_word_to_plot_df.head()
lazada_wordcloud = WordCloud(width = 1024, height = 1024, background_color = 'white', max_words = 1200
                      , min_font_size = 20).generate(str(lazada_word_to_plot))
#plot the word cloud
fig = plt.figure(figsize = (8,8), facecolor = None)
plt.imshow(lazada_wordcloud)
plt.axis('off')
plt.show()

# Lexicon

In [ ]:
toped.head(300)

In [ ]:
lexicon['number_of_words'].value_counts()

In [ ]:
cek_shopee = pd.DataFrame([])
cek_shopee['content'] = shopee['content'].copy()
cek_shopee['clean_content'] = shopee['clean_content'].copy()
cek_shopee['score'] = shopee['score'].copy()
cek_shopee['sentiment']  = shopee['sentiment'].copy() #cek sentimens sebelum diproses lexicon to sentimen
cek_shopee.sample(200)

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)
sns.kdeplot(shopee['sentiment'],color='r',shade=True)
plt.title('Sentiment Distribution')
plt.xlabel('sentiment')

In [ ]:
sns.set(style="whitegrid") 
sns.boxplot(x=shopee['sentiment'])

# Define Feature Matrix and Target Matrix

Split Training And Testing Dataset

In [58]:

X = mlbb['clean_content']     # Define feature matrix
y = mlbb['polarity']         # Define target feature matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

print('[INFO] Sebaran kelas pada training set: negatif={}, netral={}, dan positif={}'.format(sum(y_train==0), sum(y_train==1), sum(y_train==2)))
print('[INFO] Sebaran kelas pada testing set negatif={}, netral={}, dan positif={}'.format(sum(y_test==0), sum(y_test==1), sum(y_test==2)))

X_train.shape, X_test.shape, y_train.shape, y_test.shape



[INFO] Sebaran kelas pada training set: negatif=0, netral=0, dan positif=0
[INFO] Sebaran kelas pada testing set negatif=0, netral=0, dan positif=0


((544,), (234,), (544,), (234,))

Training dan Testing Tokopedia

In [59]:
X_train.head()
X_train_data=pd.DataFrame(X_train)
X_test_data=pd.DataFrame(X_test)

In [60]:
#X_test.head()
X_train_data.head()
#shopee.clean_content.head()

,clean_content
775,game good ilove
417,great would like report problem even wifi lagg...
460,absolutely love game issues face fact start pl...
705,moonton
324,game bikin stres


In [61]:
#ngram_range=(1,2) #bigram
ngram_range=(1,3) #trigram
#ngram_range=(1,2) #bigram
min_df=5
tfidf=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))

# tfidf=TfidfVectorizer(ngram_range=ngram_range,min_df=min_df)
# tfidf=TfidfVectorizer(max_features=1000)

# Term Weighting TFIDF X Training

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

In [62]:
mlbb_vector_x_train=tfidf.fit_transform(X_train).toarray()
# mlbb_vector_x_train_cv=cv.fit_transform(X_train).toarray()
mlbb_x_train_df=pd.DataFrame(mlbb_vector_x_train,columns=tfidf.get_feature_names())
# mlbb_x_train_cv_df=pd.DataFrame(mlbb_vector_x_train_cv)
# mlbb_x_train_cv_df.head()


In [48]:
mlbb_x_train_df.info()
# toped_x_train_df.info()
# lazada_x_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Columns: 8915 entries, able experience to zodiac galaxy effect
dtypes: float64(8915)
memory usage: 15.8 MB


# Term Weighting TFIDF X Testing

Tokopedia

In [63]:
mlbb_vector_x_test=tfidf.transform(X_test).toarray()
mlbb_x_test_df=pd.DataFrame(mlbb_vector_x_test,columns=tfidf.get_feature_names())
mlbb_x_test_df.head()
#toped_df.head()

,abilities gameplay,abilities gameplay match,abilities ranges,abilities ranges around,abilities weapons,abilities weapons also,able control,able control hero,able enjoy,able enjoy game,...,youtube rated,youtube rated stars,youtube suggestions,youtube suggestions players,yung,yung account,yung account bigay,zodiac,zodiac galaxy,zodiac galaxy effect
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
from sklearn.svm import SVC

# Linear Model

In [ ]:
svc_model = SVC(C= .1, kernel='linear', gamma= 1)
clf.fit(mlbb_vector_x_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = svc_model.predict(mlbb_vector_x_train)
# train_acc = accuracy_score(y_train, svc_model.predict(mlbb_vector_x_train))
print(classification_report(y_train, train_acc))
print()
print("Confusion Matrix: \n", confusion_matrix(y_train, pred))
print("Accuracy: \n", accuracy_score(y_train, pred))

# Training

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

In [65]:
C=1
# Create and train a random forest classifier
# forest = RandomForestClassifier()
models = [
          # ('k-NN', KNeighborsClassifier()),
          ('SVC linear kernel', SVC(kernel='linear',C=C,max_iter=10000)),
          ('SVC RBF kernel', SVC(kernel='rbf',gamma=1,C=C)), 
          ('SVC Polynomial (degree 3)', SVC(kernel='poly',degree=3,C=C)),
          ('SVC Sigmoid', SVC(kernel='sigmoid',C=C)),
          # ('Naive Bayes', MultinomialNB()),
          # ('Decision Tree', DecisionTreeClassifier()),
          # ('Random Forest',RandomForestClassifier(n_estimators=50)),
          # ('Regresi Linier',LogisticRegression()),
        ]


In [66]:
for name, clf in models:
  clf.fit(mlbb_vector_x_train, y_train)
  train_acc = accuracy_score(y_train, clf.predict(mlbb_vector_x_train))
  print('[INFO] Menggunakan {}, akurasi pada training set: {}'.format(name, train_acc))

[INFO] Menggunakan SVC linear kernel, akurasi pada training set: 0.9283088235294118
[INFO] Menggunakan SVC RBF kernel, akurasi pada training set: 0.9742647058823529
[INFO] Menggunakan SVC Polynomial (degree 3), akurasi pada training set: 0.9742647058823529
[INFO] Menggunakan SVC Sigmoid, akurasi pada training set: 0.9283088235294118


# Cross Validation

In [67]:
# cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

for name, model in models:
  clf.fit(mlbb_vector_x_train, y_train)
  cv = KFold(n_splits=5, random_state=27, shuffle=True)
  cross_val_acc = cross_val_score(clf, mlbb_vector_x_train, y_train, cv=cv, scoring='accuracy')
  print('[INFO] Menggunakan {}, Cross-validation akurasi: {:5f} (+/- {:5f})'.format(name, cross_val_acc.mean(), cross_val_acc.std() * 2))
  # print('[INFO] Menggunakan {}'.format(name))
  # print(classification_report(y_test, clf_predict, target_names=['negative', 'positive']))

[INFO] Menggunakan SVC linear kernel, Cross-validation akurasi: 0.478016 (+/- 0.108974)
[INFO] Menggunakan SVC RBF kernel, Cross-validation akurasi: 0.478016 (+/- 0.108974)
[INFO] Menggunakan SVC Polynomial (degree 3), Cross-validation akurasi: 0.478016 (+/- 0.108974)


KeyboardInterrupt: 

Training Confusion Matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()

# X_train = cv.fit_transform(shopee_vector_x_train.clean_content)

# X_test = cv.transform(shopee_vector_x_test.clean_content)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

for name, model in models:
  plot_confusion_matrix(model, mlbb_vector_x_train, y_train) 
  plt.title('Confusion Matrix: {}'.format(name))
  plt.show()

# Testing

In [53]:
for name, clf in models:
  clf_predict = clf.predict(mlbb_vector_x_test)
  test_acc = accuracy_score(y_test, clf.predict(mlbb_vector_x_test))
  print('[INFO] Menggunakan {}, akurasi pada testing set: {}'.format(name, test_acc))

[INFO] Menggunakan SVC linear kernel, akurasi pada testing set: 0.4935779816513762
[INFO] Menggunakan SVC RBF kernel, akurasi pada testing set: 0.5651376146788991
[INFO] Menggunakan SVC Polynomial (degree 3), akurasi pada testing set: 0.5009174311926605
[INFO] Menggunakan SVC Sigmoid, akurasi pada testing set: 0.5137614678899083


Laporan Testing

In [55]:
print("[INFO] Laporan klasifikasi\n")
for name, clf in models:
  clf_predict = clf.predict(mlbb_vector_x_test)
  print('[INFO] Menggunakan {}'.format(name))
  print(classification_report(y_test, clf_predict, target_names=['negative', 'positive']))

[INFO] Laporan klasifikasi

[INFO] Menggunakan SVC linear kernel
              precision    recall  f1-score   support

    negative       0.50      0.68      0.58       276
    positive       0.48      0.30      0.37       269

    accuracy                           0.49       545
   macro avg       0.49      0.49      0.47       545
weighted avg       0.49      0.49      0.48       545

[INFO] Menggunakan SVC RBF kernel
              precision    recall  f1-score   support

    negative       0.55      0.75      0.64       276
    positive       0.60      0.37      0.46       269

    accuracy                           0.57       545
   macro avg       0.57      0.56      0.55       545
weighted avg       0.57      0.57      0.55       545

[INFO] Menggunakan SVC Polynomial (degree 3)
              precision    recall  f1-score   support

    negative       0.50      0.95      0.66       276
    positive       0.44      0.04      0.08       269

    accuracy                          

Testing Confusion Matrix

In [ ]:
for name, model in models:
  
  plot_confusion_matrix(model, toped_vector_x_test, y_test) 
  plt.title('Confusion Matrix Test: {}'.format(name))
  
  plt.show()